In [291]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRFRegressor
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

from sklearn.utils.class_weight import compute_sample_weight

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as imb_make_pipeline

In [287]:
df_with_distance = pd.read_csv("preprocessed_data/df_insp_with_distance-v2.csv.zip")

/var/folders/_y/qzymmsdj5jj6nb7_yhb63h0r0000gn/T/ipykernel_67915/241097410.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_with_distance = pd.read_csv("preprocessed_data/df_insp_with_distance-v2.csv.zip")


In [290]:
print(df_with_distance.shape)
df_with_distance.head(10)

(148693, 33)


,Unnamed: 0,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,...,Census Tract,BIN,BBL,NTA,ClosestWorkingTankDis,ClosestWorkingTankDate,ClosestWorkingTankConfirmNum,ClosestBadTankDis,ClosestBadTankDate,ClosestBadTankConfirmNum
0,0,41678706,TITOS EMPANADAS Y MAS,Bronx,1221,JEROME AVENUE,10452.0,9178193056,Latin American,06/03/2019,...,21900.0,2003083.0,2.025060e+09,BX26,0.331346,09/09/2020,WTI7992302276,5.422600,01/01/2021,WTI1718389770
1,1,50065644,NEW GRAND BUFFET,Queens,3445,48TH ST,11101.0,7182788889,Chinese,05/18/2019,...,17100.0,4000822.0,4.001200e+09,QN31,0.425275,08/25/2020,WTI6227302551,0.659718,01/01/2021,WTI1718389770
2,2,41029356,THE DUPLEX,Manhattan,61,CHRISTOPHER STREET,10014.0,2122555438,American,12/17/2019,...,7300.0,1082224.0,1.006100e+09,MN23,0.051459,11/02/2020,WTI8403171362,1.463683,07/28/2015,WTI8645897156
3,3,50058634,LA FLOR DEL CANARIO #2 RESTAURANT PIZZERIA,Brooklyn,2225,PITKIN AVENUE,11207.0,7184855000,Spanish,11/21/2017,...,115200.0,3088794.0,3.039940e+09,BK82,0.849993,10/14/2020,WTI0264803420,6.166759,01/01/2021,WTI1718389770
4,4,50044708,21 BAR,Manhattan,4318,BROADWAY,10033.0,9179752690,American,06/13/2018,...,27100.0,1063931.0,1.021640e+09,MN35,0.265592,10/28/2020,WTI1137511076,6.500742,01/01/2021,WTI1718389770
5,5,41549759,CHIPOTLE MEXICAN GRILL,Manhattan,235,WEST 56 STREET,10019.0,6466266520,Tex-Mex,02/13/2017,...,13700.0,1070014.0,1.010288e+09,MN17,0.000452,10/06/2020,WTI8515487754,0.963038,05/08/2015,WTI5981923701
6,6,41426561,BROADWAY BAKERY,Queens,8109,41 AVENUE,11373.0,3472578759,Bakery Products/Desserts,07/05/2018,...,26901.0,4307358.0,4.014928e+09,QN29,0.302896,08/20/2020,WTI4449194756,2.311711,01/01/2021,WTI1718389770
7,7,50106966,SUSHI MARKET,Queens,24825,NORTHERN BLVD,11362.0,3474084170,Japanese,08/10/2021,...,147900.0,4169059.0,4.081180e+09,QN45,2.351889,01/27/2020,WTI3755708068,9.814211,01/01/2021,WTI1718389770
8,8,41112470,VILLA MIA PIZZA,Brooklyn,526,3 AVENUE,11215.0,7187883330,Pizza,11/21/2019,...,11700.0,3023074.0,3.010260e+09,BK37,0.542898,06/25/2020,WTI7811492225,3.222641,07/28/2015,WTI8645897156
9,9,41611664,SERVICE BAR 6,Queens,110-00,ROCKAWAY BOULEVARD,11420.0,7182153311,American,12/07/2016,...,86400.0,4457718.0,4.115430e+09,QN55,1.674951,05/30/2020,WTI9603835434,7.511339,01/01/2021,WTI1718389770


In [316]:
df = df_with_distance.copy()
# Convert A, B, C to 3, 2, 1 for encodings, drop all other ratings
grades = ["A", "B", "C"]
df = df[df['GRADE'].isin(grades)]
df['SCORE'] = df.loc[:, 'GRADE']
df.loc[df["SCORE"] == "A", "GRADE"] = 3
df.loc[df["SCORE"] == "B", "GRADE"] = 2
df.loc[df["SCORE"] == "C", "GRADE"] = 1

# Replace violation code, zipcode and cuisine type with its average grade:
target = df["GRADE"]
te = TargetEncoder(cols=["VIOLATION CODE", "ZIPCODE", "CUISINE DESCRIPTION"])
te.fit(df, target)
df = te.transform(df)

# Peform OHE on Boro and Critical Flag:
df = pd.get_dummies(df, columns=["BORO"])
df = pd.get_dummies(df, columns=["CRITICAL FLAG"])

print(df.columns)
df = df.drop(["DBA", 
              "BUILDING", 
              "STREET",
              "PHONE",
              "ACTION",
              "VIOLATION DESCRIPTION",
              "Latitude",
              "Longitude",
              "Community Board",
              "Council District",
              "Census Tract", 
              "RECORD DATE",
              "INSPECTION DATE",
              "INSPECTION TYPE",
              "CRITICAL FLAG_Critical",
              "BIN",
              "BBL",
              "SCORE",
              "NTA",
              'Unnamed: 0'], axis=1)
df = df.dropna()
df.head()

Index(['Unnamed: 0', 'CAMIS', 'DBA', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'SCORE', 'GRADE', 'GRADE DATE', 'RECORD DATE',
       'INSPECTION TYPE', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA',
       'ClosestWorkingTankDis', 'ClosestWorkingTankDate',
       'ClosestWorkingTankConfirmNum', 'ClosestBadTankDis',
       'ClosestBadTankDate', 'ClosestBadTankConfirmNum', 'BORO_Bronx',
       'BORO_Brooklyn', 'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island',
       'CRITICAL FLAG_Critical', 'CRITICAL FLAG_Not Critical'],
      dtype='object')


,CAMIS,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,GRADE DATE,ClosestWorkingTankDis,ClosestWorkingTankDate,ClosestWorkingTankConfirmNum,ClosestBadTankDis,ClosestBadTankDate,ClosestBadTankConfirmNum,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical
0,41678706,2.622771,2.675366,2.597904,2,06/03/2019,0.331346,09/09/2020,WTI7992302276,5.422600,01/01/2021,WTI1718389770,1,0,0,0,0,0
1,50065644,2.797419,2.714226,2.665241,2,05/18/2019,0.425275,08/25/2020,WTI6227302551,0.659718,01/01/2021,WTI1718389770,0,0,0,1,0,0
2,41029356,2.784406,2.834724,2.660336,3,12/17/2019,0.051459,11/02/2020,WTI8403171362,1.463683,07/28/2015,WTI8645897156,0,0,1,0,0,1
3,50058634,2.653798,2.609433,2.807527,3,11/21/2017,0.849993,10/14/2020,WTI0264803420,6.166759,01/01/2021,WTI1718389770,0,1,0,0,0,0
4,50044708,2.607099,2.834724,2.785281,3,06/13/2018,0.265592,10/28/2020,WTI1137511076,6.500742,01/01/2021,WTI1718389770,0,0,1,0,0,0


In [317]:
df.columns.values

array(['CAMIS', 'ZIPCODE', 'CUISINE DESCRIPTION', 'VIOLATION CODE',
       'GRADE', 'GRADE DATE', 'ClosestWorkingTankDis',
       'ClosestWorkingTankDate', 'ClosestWorkingTankConfirmNum',
       'ClosestBadTankDis', 'ClosestBadTankDate',
       'ClosestBadTankConfirmNum', 'BORO_Bronx', 'BORO_Brooklyn',
       'BORO_Manhattan', 'BORO_Queens', 'BORO_Staten Island',
       'CRITICAL FLAG_Not Critical'], dtype=object)

In [310]:
# check whether CAMIS in unique ID
print("Number of restaurant: ", pd.unique(df['CAMIS']).shape)
print("Dataframe shape: ", df.shape)

Number of restaurant:  (22527,)
Dataframe shape:  (144130, 18)


In [329]:
# transform to datetime object
def transform_to_datetime(df, colname:str, format_str:str = '%m/%d/%Y'):
    import datetime as dt 
    return [dt.datetime.strptime(str(date_string), format_str) for date_string in df[colname].values ]
df['GRADE DATE'] = transform_to_datetime(df, 'GRADE DATE')
df['ClosestWorkingTankDate'] = transform_to_datetime(df, 'ClosestWorkingTankDate')
df['ClosestBadTankDate'] = transform_to_datetime(df, 'ClosestBadTankDate')

# Find records after 2019
df.drop_duplicates(inplace=True)
print("Number of records after 2019: ", sum(df['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')))
df.reset_index(drop = True, inplace= True)
df['GRADE'] = df['GRADE'].astype(float)

Number of records after 2019:  87939


In [330]:
print("Most recent grade date: ", df['GRADE DATE'].max())
print("Earliest grade date: ", df['GRADE DATE'].min())
print("Most recent ClosestWorkingTankDate: ", df['ClosestWorkingTankDate'].max())
print("Earliest ClosestWorkingTankDate: ", df['ClosestWorkingTankDate'].min())
print("Most recent ClosestBadTankDate: ", df['ClosestBadTankDate'].max())
print("Earliest ClosestBadTankDate: ", df['ClosestBadTankDate'].min())

Most recent grade date:  2022-04-08 00:00:00
Earliest grade date:  2015-05-14 00:00:00
Most recent ClosestWorkingTankDate:  2020-12-31 00:00:00
Earliest ClosestWorkingTankDate:  2020-01-07 00:00:00
Most recent ClosestBadTankDate:  2021-01-01 00:00:00
Earliest ClosestBadTankDate:  2015-05-08 00:00:00


# Average inspection by restaurant by date

In [336]:
# Average inspection by restaurant by date
df_date = df[['CAMIS', 'GRADE DATE','ClosestWorkingTankDate', 'ClosestBadTankDate']].drop_duplicates().sort_values(by = ['CAMIS', 'GRADE DATE'])
df_grouped = df.drop(columns=['ClosestWorkingTankDate', 'ClosestBadTankDate']).groupby(['CAMIS', 'GRADE DATE']).mean().reset_index()

df_grouped = pd.concat([df_grouped, df_date.drop(columns=['GRADE DATE','CAMIS'])], axis= 1)
print(df_grouped.shape)
df_grouped.head()

# TODO: Recent closest working/bad tank date before grade date?


(74124, 16)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate
0,30075445.0,2019-06-11,2.876777,2.778831,2.847154,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,1.000000,2020-09-09,2021-01-01
1,30075445.0,2019-11-04,2.876777,2.778831,2.621810,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-08-25,2021-01-01
2,30075445.0,2021-08-10,2.876777,2.778831,2.785281,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-11-02,2015-07-28
3,30112340.0,2019-03-04,2.801389,2.913240,2.792569,3.0,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.666667,2020-10-14,2021-01-01
4,30112340.0,2021-08-05,2.801389,2.913240,2.634818,2.0,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.333333,2020-10-28,2021-01-01


In [340]:
df_grouped.isnull().sum()

CAMIS                         17675
GRADE DATE                    17675
ZIPCODE                       17675
CUISINE DESCRIPTION           17675
VIOLATION CODE                17675
GRADE                         17675
ClosestWorkingTankDis         17675
ClosestBadTankDis             17675
BORO_Bronx                    17675
BORO_Brooklyn                 17675
BORO_Manhattan                17675
BORO_Queens                   17675
BORO_Staten Island            17675
CRITICAL FLAG_Not Critical    17675
ClosestWorkingTankDate        17675
ClosestBadTankDate            17675
dtype: int64

# Find previous inspection

In [337]:
# Calculate recency: recent visit - previous visit
camis = df_grouped['CAMIS'].unique()
new_df = pd.DataFrame()
for idx in camis:
    # filter the records of specific restuarant using CAMIS
    subset = df_grouped[df_grouped.CAMIS == idx].sort_values(by = 'GRADE DATE')
    if subset.shape[0] == 1:
        continue
    prev_df = pd.DataFrame(index=subset.index.values)
    prev_df['GRADE DATE'] = subset['GRADE DATE']
    # prev_df['PREV_CRITICAL FLAG_Not Applicable'] = subset.shift(1)['CRITICAL FLAG_Not Applicable']
    # prev_df['PREV_CRITICAL FLAG_Not Critical'] = subset.shift(1)['CRITICAL FLAG_Not Critical']
    # find previous grading
    
    # find previous grading date and RECENCY
    df_date = subset[['GRADE', 'GRADE DATE']].drop_duplicates()
    if df_date.shape[0] != df_date['GRADE DATE'].unique().shape[0]:
        print(idx)
        break
    df_date['PREV_GRADE'] = df_date.shift(1)['GRADE']
    df_date['PREV_GRADE DATE'] = df_date.shift(1)['GRADE DATE']
    df_date['RECENCY'] = df_date['GRADE DATE'] - df_date.shift(1)['GRADE DATE']
    prev_df = prev_df.merge(df_date.drop(columns='GRADE'), how = 'left', on='GRADE DATE').set_index(subset.index.values)
    # append to new dataframe
    new_df = pd.concat([new_df, prev_df], axis=0)

In [338]:
# Filter rows after year 2019
df_concat = pd.concat([df_grouped, new_df.drop(columns=['GRADE DATE'])], axis=1)
# filter rows with date
df_concat_filtered = df_concat[df_grouped['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')]
print(df_concat_filtered.shape)
df_concat_filtered.head(10)

(36880, 19)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,ClosestWorkingTankDate,ClosestBadTankDate,PREV_GRADE,PREV_GRADE DATE,RECENCY
0,30075445.0,2019-06-11,2.876777,2.778831,2.847154,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,1.000000,2020-09-09,2021-01-01,NaN,NaT,NaT
1,30075445.0,2019-11-04,2.876777,2.778831,2.621810,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-08-25,2021-01-01,3.0,2019-06-11,146 days
2,30075445.0,2021-08-10,2.876777,2.778831,2.785281,3.0,0.484806,7.207183,1.0,0.0,0.0,0.0,0.0,0.000000,2020-11-02,2015-07-28,3.0,2019-11-04,645 days
3,30112340.0,2019-03-04,2.801389,2.913240,2.792569,3.0,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.666667,2020-10-14,2021-01-01,NaN,NaT,NaT
4,30112340.0,2021-08-05,2.801389,2.913240,2.634818,2.0,0.226460,4.392523,0.0,1.0,0.0,0.0,0.0,0.333333,2020-10-28,2021-01-01,3.0,2019-03-04,885 days
5,30191841.0,2019-06-06,2.831125,2.814634,2.857626,3.0,0.004614,1.120030,0.0,0.0,1.0,0.0,0.0,0.500000,2020-10-06,2015-05-08,NaN,NaT,NaT
6,30191841.0,2022-01-04,2.831125,2.814634,2.870896,3.0,0.004614,1.120030,0.0,0.0,1.0,0.0,0.0,0.666667,2020-08-20,2021-01-01,3.0,2019-06-06,943 days
7,40356018.0,2019-11-08,2.868895,2.834724,2.855589,3.0,0.315359,9.367778,0.0,1.0,0.0,0.0,0.0,0.666667,2020-01-27,2021-01-01,NaN,NaT,NaT
8,40356018.0,2022-02-01,2.868895,2.834724,2.622308,3.0,0.315359,9.367778,0.0,1.0,0.0,0.0,0.0,0.000000,2020-06-25,2015-07-28,3.0,2019-11-08,816 days
9,40356483.0,2019-05-03,2.794118,2.770759,2.854308,3.0,2.320265,8.494743,0.0,1.0,0.0,0.0,0.0,0.666667,2020-05-30,2021-01-01,NaN,NaT,NaT


In [339]:
# Null values in each column
df_concat_filtered.isnull().sum()

CAMIS                             0
GRADE DATE                        0
ZIPCODE                           0
CUISINE DESCRIPTION               0
VIOLATION CODE                    0
GRADE                             0
ClosestWorkingTankDis             0
ClosestBadTankDis                 0
BORO_Bronx                        0
BORO_Brooklyn                     0
BORO_Manhattan                    0
BORO_Queens                       0
BORO_Staten Island                0
CRITICAL FLAG_Not Critical        0
ClosestWorkingTankDate        11699
ClosestBadTankDate            11699
PREV_GRADE                    12556
PREV_GRADE DATE               12556
RECENCY                       12556
dtype: int64

# Frequency of inspection for each restaurant after 2019

In [300]:
# Calculate frequency: time of grading within three year
df_freq = pd.DataFrame(df_concat_filtered.groupby(['CAMIS']).size(), columns=["frequency"])
print(df_freq.frequency.unique())
print(df_freq.head(10))
# Inspection is not once a year
df_concat_filtered[df_concat_filtered.CAMIS == 41319107]

[3 2 1 4 5]
          frequency
CAMIS              
30075445          3
30112340          2
30191841          2
40356018          2
40356483          2
40356731          2
40357217          2
40359480          1
40359705          2
40360045          2


,CAMIS,GRADE DATE,Unnamed: 0,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,ClosestWorkingTankDis,ClosestBadTankDis,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Critical,PREV_GRADE,PREV_GRADE DATE,RECENCY
9885,41319107,2019-01-28,33058.333333,2.670737,2.834724,2.731677,3.0,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,NaN,NaT,NaT
9886,41319107,2019-05-30,94696.000000,2.670737,2.834724,2.709183,3.0,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.000000,3.0,2019-01-28,122 days
9887,41319107,2019-09-23,50979.666667,2.670737,2.834724,2.784434,3.0,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,3.0,2019-05-30,116 days
9888,41319107,2020-01-30,63468.750000,2.670737,2.834724,2.646220,3.0,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.500000,3.0,2019-09-23,129 days
9889,41319107,2022-01-27,63936.666667,2.670737,2.834724,2.729957,3.0,0.102618,4.905618,0.0,0.0,0.0,1.0,0.0,0.666667,3.0,2020-01-30,728 days
